In [ ]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

apiKey = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=apiKey)

print(apiKey)

In [ ]:
GPT_MODEL = "gpt-3.5-turbo-0613"
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def execute_function_call(message):
    paramFn = json.loads(message["function_call"]["arguments"])
    results = globals()[message["function_call"]["name"]](paramFn)

    return results


def establish_context(system_message, user_message, functions=None, messages=[],
                      user_id='test_user'):
    if len(messages) == 0:
        messages.append({"role": "system", "content": system_message})
        Message.create(user_id=user_id, role='system', message=system_message, message_gpt=system_message)

    messages.append({"role": "user", "content": user_message})
    Message.create(user_id=user_id, role='user', message=user_message, message_gpt=user_message)
    chat_response = chat_completion_request(messages, functions)
    print('chat_response', chat_response.json())
    assistant_message = chat_response.json()["choices"][0]["message"]
    messages.append(assistant_message)
    if assistant_message.get("function_call"):
        results = execute_function_call(assistant_message)
        messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
        Message.create(user_id=user_id, role='user', message=results, message_gpt=results.get('message'))

    return messages


functionsStock = [
    {
        "name": "forward_action",
        "description": "Lấy mã stock từ message",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_key": {
                    "type": "string",
                    "description": "Các mã stock của các công ty ở Việt Nam",
                },
                "action": {
                    "type": "string",
                    "enum": ["dự đoán", "so sánh", "báo cáo tài chính", "thông tin cơ bản"],
                    "description": "Lấy ra hành động mà người dùng mong muốn thực hiện",
                },
                "num_days": {
                    "type": "integer",
                    "description": "Số ngày dự đoán",
                },
                "model": {
                    "type": "string",
                    "enum": ["rnn", "linear"],
                    "description": "Mô hình dự đoán",
                }
            },
            "required": ["location", "format"],
        },
    },
]